<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-data-set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del Dataset

Durante este proceso se crea un dataset con lo datos procesados y listos para el entrenamiento del modelo.

# Parametros

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.
* dataset_path: ruta en el que se almacenará el dataset
* labels_file: ruta en la que se almacenará la lista de etiquetas

In [1]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Datos de origen
ruta_archivo_GoldStandardTrainingEA = ruta_base + 'GoldStandardTrainingEAs.txt'
ruta_archivo_GoldStandardTestEA = ruta_base + 'GoldStandardTestEAs.txt'
ruta_archivos_entrenamiento = ruta_base + 'Training/'
ruta_archivos_test = ruta_base + 'Test/'

#Dataset procesado
ruta_dataset = ruta_base + 'Dataset'

In [2]:
!pip install -q datasets unidecode nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.3 MB/s eta 0:00:00


Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [3]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


#Creación del dataset




Se crea un dataset con los tres segmentos: train, validation y test. Las etiquetas de train y validation se toman del fichero GoldStandardTrainingEAs.txt y las etiquetas del segmento test se toman del fichero GoldStandardTestEAs.txt.

Se cargan los archivos para su tratamiento.

In [4]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    ruta_archivo_GoldStandardTrainingEA, sep=' ', header=None, names=column_names)
df_goldStandardTestEAs = pd.read_csv(
    ruta_archivo_GoldStandardTestEA, sep=' ', header=None, names=column_names)

Generación de las etiquetas

In [5]:
df_goldStandardTrainingEAs

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27690409,T50.2X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24626366,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27967924,O90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25774939,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24878023,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14497,24368169,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14498,26850773,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,26850775,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14500,26850771,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se unen todos los códigos EAs de todas las columnas para conocer cuántos códigos EAs hay representados en los casos clínicos.

In [40]:
def crear_df_cantidad(dataframe):
  ds_cantidad_TieneEA_EA1 = dataframe.TieneEA_EA1.value_counts(dropna=True, sort=True)
  ds_cantidad_EA2 = dataframe.EA2.value_counts(dropna=True, sort=True)
  ds_all = ds_cantidad_TieneEA_EA1.append(ds_cantidad_EA2)
  ds_cantidad_EA3 = dataframe.EA3.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA3)
  ds_cantidad_EA4 = dataframe.EA4.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA4)
  ds_cantidad_EA5 = dataframe.EA5.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA5)
  ds_cantidad_EA6 = dataframe.EA6.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA6)
  ds_cantidad_EA7 = dataframe.EA7.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA7)
  ds_cantidad_EA8 = dataframe.EA8.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA8)
  ds_cantidad_EA9 = dataframe.EA9.value_counts(dropna=True, sort=True)
  ds_all = ds_all.append(ds_cantidad_EA9)

  df_cantidad = pd.DataFrame(ds_all)
  df_cantidad = df_cantidad.reset_index()
  df_cantidad.columns = ['eas', 'counts'] 
  return df_cantidad

df_cantidad = crear_df_cantidad(df_goldStandardTrainingEAs)
df_cantidad.count()

<ipython-input-40-3c8a9a3daee1>:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_cantidad_TieneEA_EA1.append(ds_cantidad_EA2)
<ipython-input-40-3c8a9a3daee1>:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_all.append(ds_cantidad_EA3)
<ipython-input-40-3c8a9a3daee1>:8: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_all.append(ds_cantidad_EA4)
<ipython-input-40-3c8a9a3daee1>:10: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_all.append(ds_cantidad_EA5)
<ipython-input-40-3c8a9a3daee1>:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future vers

eas       789
counts    789
dtype: int64

In [45]:
from datasets import ClassLabel

def obtener_lista_actos(dataframe, rows):
  actos = dataframe.query("Acto == 0")
  for ea in rows["eas"]:
    actos = actos.append(obtener_actos(dataframe, ea), ignore_index=True)
  return actos

def obtener_actos(dataframe, EA_code):
  query = ("TieneEA_EA1 == '{EA}'")
  query = query.replace ('{EA}', EA_code)
  return dataframe.query(query)
  
df_entrenamiento_codigos_mas_frecuentes = obtener_lista_actos(
    df_goldStandardTrainingEAs, df_cantidad.query("eas != 'NONE'").head(8))

df_test_codigos_mas_frecuentes = obtener_lista_actos(
    df_goldStandardTestEAs, df_cantidad.query("eas != 'NONE'").head(8))

def crear_lista_etiquetas(dataframe):
  label_list = list(dataframe.TieneEA_EA1.unique())
  label_list.extend(list(dataframe.EA2.unique()))
  label_list.extend(list(dataframe.EA3.unique()))
  label_list.extend(list(dataframe.EA4.unique()))
  label_list.extend(list(dataframe.EA5.unique()))
  label_list.extend(list(dataframe.EA6.unique()))
  label_list.extend(list(dataframe.EA7.unique()))
  label_list.extend(list(dataframe.EA8.unique()))
  label_list.extend(list(dataframe.EA9.unique()))
  label_list.extend(list(dataframe.EA10.unique()))
  df = pd.DataFrame(label_list, columns=["label"])
  df = df.dropna()
  label_list = list(df.label.unique())
  return ClassLabel(num_classes=len(label_list), names=label_list)

class2label_entrenamiento = crear_lista_etiquetas(df_goldStandardTrainingEAs)
class2label_entrenamiento_masFrecuentes = crear_lista_etiquetas(df_entrenamiento_codigos_mas_frecuentes)
class2label_test = crear_lista_etiquetas(df_goldStandardTestEAs)
class2label_test_masFrecuentes = crear_lista_etiquetas(df_test_codigos_mas_frecuentes)

<ipython-input-45-117430a667cf>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actos = actos.append(obtener_actos(dataframe, ea), ignore_index=True)
<ipython-input-45-117430a667cf>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actos = actos.append(obtener_actos(dataframe, ea), ignore_index=True)


In [47]:
df_entrenamiento_codigos_mas_frecuentes

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27820190,T38.0X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24640635,T38.0X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27725771,T38.0X5A,T45.1X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27834890,T38.0X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26111542,T38.0X5A,T46.5X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
706,25078241,Y83.1,T82.03XA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707,27184410,Y83.1,T82.9XXA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,26332930,Y83.1,T84.84XA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,24008892,Y83.1,T82.857D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df_entrenamiento_codigos_mas_frecuentes.TieneEA_EA1.unique()

array(['T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA',
       'T45.1X5A', 'Y83.1'], dtype=object)

In [46]:
class2label_entrenamiento_masFrecuentes

ClassLabel(names=['T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A', 'Y83.1', 'T46.5X5A', 'E09.9', 'E24.2', 'T83.511A', 'T40.4X5A', 'T80.1XXA', 'T42.1X5A', 'G72.0', 'T43.215A', 'D70.1', 'T37.5X5A', 'E66.1', 'E27.3', 'T45.525A', 'T79.A21A', 'D75.82', 'T36.1X5A', 'T81.718A', 'T36.8X5A', 'Y65.8', 'T81.9XXS', 'T46.4X5A', 'T46.1X5A', 'T50.0X5A', 'T43.225A', 'I95.2', 'T82.897A', 'T88.9XXS', 'Y84.6', 'T80.211A', 'M96.1', 'T40.2X5A', 'T38.0X5S', 'T38.3X5A', 'G21.19', 'K91.89', 'T46.0X5A', 'T86.12', 'T79.6XXA', 'T84.7XXA', 'T86.11', 'P03.2', 'P01.3', 'Y83.6', 'T87.43', 'T85.84XA', 'T81.83XA', 'L27.1', 'Y83.9', 'Y83.8', 'T81.31XA', 'T80.89XA', 'T50.3X5A', 'K66.0', 'T36.95XA', 'Y83.2', 'L76.22', 'I97.89', 'I97.711', 'D70.2', 'L76.34', 'T50.8X5A', 'T84.53XA', 'T36.7X5A', 'D61.810', 'D64.81', 'E89.0', 'R50.2', 'T85.79XA', 'T82.03XA', 'T84.52XA', 'T82.7XXA', 'T82.855A', 'I97.410', 'T84.84XA', 'T84.030A', 'T84.021A', 'T84.51XA', 'T82.867A', 'T84.228A', 'T85.29XA', 'T82.190A',

Precesamiento del texto de los informes

In [36]:
import os
import re
from unidecode import unidecode
from nltk.corpus import stopwords 
import nltk
import json

JUICIO_CLINICO = 'juicio clinico'
JUICIO_DIAGNOSTICO = 'juicio diagnostico'
TRATAMIENTO = 'Tratamiento'
FIRMADO_XXX = '_FIRMADO_XXXX_'
ETIQUETAS = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta", 
             "analitica", "comentarios", "juicio","clinico","tratamiento",
             "recomendaciones","exploraciones", "complementarias"]

re_sexo = re.compile(r'sexo [a-z]*')
re_juicio1 = re.compile(r'juicio [a-z]*')
re_juicio2 = re.compile(r'j[a-z]* clinico')
re_gestacion = re.compile(r'gestacion [a-z]*')

campos = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta"]

apartados_de_interes = [
    "jdtco", "jc"] #, "sexo", "comentarios"
apartado_sexo = "sexo"

apartados_descartados = [
    "recomendaciones", "lopd", "pagina", "datos personales seran tratados"]    

TRAINING = 0
TEST = 1
TRAINING_MAS_FRECUENTES = 2
TEST_MAS_FRECUENTES = 3

nltk.download('stopwords')

def omitir_textos_anonimizacion (linea):
  # busca expresiones del tipo _HOSPITAL_XXXX_ _DIRECCION_XXXX_
  return re.sub (r'_\w+_XXXX_', '', linea)

def omitir_no_texto (linea):
  return re.sub (r'[^\w]', ' ', linea)

def omitir_numeros (linea):
  return re.sub(r'[0-9]+', ' ', linea)

def es_linea_comienzo_seleccion (linea):
  return normalizar_texto(linea.strip()) == JUICIO_CLINICO

def es_linea_fin_seleccion (linea):
  return FIRMADO_XXX in linea.strip()

def linea_tiene_contenido (linea):
  return linea.strip() != ''

def es_apartado_interes (linea, incluir_todo):
  apartados = apartados_de_interes
  if (incluir_todo):
    apartados.append(apartado_sexo)
  for apartado in apartados:
    if (apartado in linea):
      return True
  return False

def es_apartado (linea, incluir_todo):
  return (re_juicio1.match (linea) != None or
          re_juicio2.match (linea) != None or
          re_gestacion.match (linea) != None or
          es_apartado_interes (linea, incluir_todo))

def es_apartado_descartado (linea):
  for apartado in apartados_descartados:
    if (apartado in linea):
      return True
  return False

def detectar_etiquetas (linea):
  words = linea.strip().split()
  if len(words) >= 1 and len(words) <= 3:
      return linea
  return ""      

def omitir_stopwors (linea):
  stop_words = set(stopwords.words('spanish')) 
  palabras = linea.split() 
  resultado = ''
  for paralabra in palabras: 
      if not resultado in stop_words: 
        resultado += ' ' + paralabra
  return resultado

def normalizar_texto (linea):
  linea = unidecode(linea) 
  linea = linea.lower().strip()
  return linea

def quitar_etiquetas (linea):
  for etiqueta in ETIQUETAS:
    linea = linea.replace(etiqueta, "")
  return linea

def limpiar_texto (informe_medico, diccionario_apartados, incluir_todo):
  texto = ''
  seleccionar = False  
  apartado = ""
  texto_apartado = ""

  for lineaTexto in informe_medico:
    lineaTexto = omitir_textos_anonimizacion(lineaTexto)
    lineaTexto = normalizar_texto(lineaTexto)
    lineaTexto = omitir_no_texto(lineaTexto)
    lineaTexto = omitir_numeros(lineaTexto)
    if (not incluir_todo and apartado_sexo in lineaTexto):
      linea_array = lineaTexto.split(" ")      
      diccionario_apartados[apartado_sexo] = linea_array[len(linea_array)-1]
    elif (es_apartado(lineaTexto, incluir_todo)):
      if (apartado != "" and apartado != lineaTexto):
        diccionario_apartados[apartado] = texto_apartado
        texto_apartado = ""
      apartado = lineaTexto   
      seleccionar = True   
    elif (es_apartado_descartado(lineaTexto)):        
        if (texto_apartado != ""):
          diccionario_apartados[apartado] = texto_apartado
        apartado = "" 
        texto_apartado = ""
        seleccionar = False   
    if (seleccionar):      
      lineaTexto = omitir_stopwors(lineaTexto)  
      texto_apartado += lineaTexto 

  if (texto_apartado != ""):
    diccionario_apartados[apartado] = texto_apartado

  for apartado in diccionario_apartados.values():
    texto += apartado

  json_apartados = [ {'apartado' : k, 'texto' : diccionario_apartados[k]} for k in diccionario_apartados]
  return texto, json_apartados

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location, class2label):  
  texto_acto = ''  
  diccionario_apartados = {}
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico:
      resultado = limpiar_texto(informe_medico, diccionario_apartados, False)
    if (resultado[0].strip() == ""):
      with open (files_location + nombre_informe, 'rt') as informe_medico:
        resultado = limpiar_texto(informe_medico, diccionario_apartados, True)
    texto_acto = texto_acto + resultado[0]
    texto_acto = texto_acto + "\n"
  label_list_int = convertir_codigos_EA_a_entero(class2label, label_list)
  return [acto,
          label_list_int[0], 
          label_list[0], 
          label_list_int,
          label_list, 
          lista_nombres_informes_medicos, 
          texto_acto, 
          json.dumps(resultado[1])]

def convertir_codigos_EA_a_entero(class2label, label_list):
  label_list_int = list()
  for codigo_ea in label_list:
    label_list_int.append(class2label.str2int(codigo_ea))
  return label_list_int

def tomar_codigos_EA_de_todas_las_columnas(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col].strip())
  return label_list

def crear_vector_datos (dataframe, file_list, files_location, class2label, cantidad): 
  ds_resultado = []
  i = 0
  for index, row in dataframe.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    ds_resultado.append(obtener_texto(row['Acto'], tomar_codigos_EA_de_todas_las_columnas(row), 
                                      list(archivos_acto), files_location, class2label))
    i += 1
    if (cantidad > 0 and i > cantidad):
     break
  return ds_resultado

def crear_dataframe (labeled_array):
  return pd.DataFrame(labeled_array, 
                      columns=['acto', 
                               'label', 'label_str', 
                               'label_list', 'label_list_str', 
                               'informes', 
                               'text', 'json'])

def crear_dataframe_para_un_informe (nombre_archivo_informe):
  acto = nombre_archivo_informe.split("-")[0]
  labeled_array_training = crear_vector_datos (
      df_goldStandardTrainingEAs.query("Acto == " + acto), [nombre_archivo_informe], 
        ruta_archivos_entrenamiento, class2label_entrenamiento, 0)
  return crear_dataframe (labeled_array_training)  

def crear_dataframe_train_test_masFrecuentes ():
  lista_archivos_entrenamiento = [f for f in os.listdir(ruta_archivos_entrenamiento) 
                      if os.path.isfile(os.path.join(ruta_archivos_entrenamiento, f))]

  vector_datos_entrenamiento = crear_vector_datos (
      df_goldStandardTrainingEAs, lista_archivos_entrenamiento, 
      ruta_archivos_entrenamiento, class2label_entrenamiento, 0)
  df_entrenamiento = crear_dataframe (vector_datos_entrenamiento)
  
  vector_datos_entrenamiento_masFrecuentes = crear_vector_datos (
      df_entrenamiento_codigos_mas_frecuentes, lista_archivos_entrenamiento, 
      ruta_archivos_entrenamiento, class2label_entrenamiento, 0)
  df_entrenamiento_masFrecuentes = crear_dataframe (vector_datos_entrenamiento_masFrecuentes)
  
  lista_archivos_test = [f for f in os.listdir(ruta_archivos_test) 
                  if os.path.isfile(os.path.join(ruta_archivos_test, f))]

  vector_datos_test = crear_vector_datos (
      df_goldStandardTestEAs, lista_archivos_test, ruta_archivos_test, class2label_test, 0)
  df_test = crear_dataframe (vector_datos_test)

  vector_datos_test_mas_frecuentes = crear_vector_datos (
      df_test_codigos_mas_frecuentes, lista_archivos_test, ruta_archivos_test, class2label_test, 0)
  df_test_masFrecuentes = crear_dataframe (vector_datos_test_mas_frecuentes)

  return df_entrenamiento, df_test, df_entrenamiento_masFrecuentes, df_test_masFrecuentes

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
crear_dataframe_para_un_informe("23062488-162179798.txt")

,acto,label,label_str,label_list,label_list_str,informes,text,json
0,23062488,33,T85.71XA,"[33, 184]","[T85.71XA, T82.898A]",[23062488-162179798.txt],hombre juicio clinico exitus sepsis de origen ...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."


In [37]:
dataframes = crear_dataframe_train_test()

In [38]:
dataframes[0]

,acto,label,label_str,label_list,label_list_str,informes,text,json
0,27690409,0,T50.2X5A,[0],[T50.2X5A],[27690409-171690541.txt],hombre juicio clinico primer episodio de desco...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."
1,24626366,1,NONE,[1],[NONE],[24626366-163054389.txt],mujer juicio clinico neumonia probablemente ne...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."
2,27967924,2,O90.0,[2],[O90.0],[27967924-172443607.txt],gestacion actual f u r f p p gestacion unica ...,"[{""apartado"": ""gestacion actual"", ""texto"": "" g..."
3,25774939,1,NONE,[1],[NONE],[25774939-166138278.txt],hombre juicio clinico episodio de icc leve pre...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."
4,24878023,1,NONE,[1],[NONE],[24878023-163476964.txt],hombre juicio clinico celulitis perirobitaria ...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."
...,...,...,...,...,...,...,...,...
14497,24368169,1,NONE,[1],[NONE],[24368169-162102373.txt],mujer juicio clinico anemizacion aguda sobre a...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."
14498,26850773,1,NONE,[1],[NONE],"[26850773-169269452.txt, 26850773-169270850.txt]",juicio clinico cancer de mama derecha anteced...,"[{""apartado"": ""juicio clinico"", ""texto"": "" jui..."
14499,26850775,1,NONE,[1],[NONE],[26850775-169239813.txt],mujer juicio clinico intolerancia a aines asma...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."
14500,26850771,1,NONE,[1],[NONE],[26850771-169251366.txt],mujer juicio diagnostico multiples polipos col...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."


Composición del dataset

In [39]:
from datasets import Dataset, Value, DatasetDict

def set_dataset_features (dataset, class2label):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
dataset_entrenamiento = Dataset.from_pandas(dataframes[TRAINING])
dataset_entrenamiento = set_dataset_features (dataset_entrenamiento, class2label_entrenamiento)
dataset_entrenamiento_validacion = dataset_entrenamiento.train_test_split(test_size=0.1)

dataset_entrenamiento_masFrecuentes = Dataset.from_pandas(dataframes[TRAINING_MAS_FRECUENTES])
dataset_entrenamiento_masFrecuentes = set_dataset_features (dataset_entrenamiento_masFrecuentes, class2label_entrenamiento_masFrecuentes)
dataset_entrenamiento_validacion_masFrecuentes = dataset_entrenamiento_masFrecuentes.train_test_split(test_size=0.1)

dataset_test = Dataset.from_pandas(dataframes[TEST])
dataset_test = set_dataset_features (dataset_test, class2label_test)

dataset_test_masFrecuentes = Dataset.from_pandas(dataframes[TEST_MAS_FRECUENTES])
dataset_test_masFrecuentes = set_dataset_features (dataset_test_masFrecuentes, class2label_test_masFrecuentes)

dataset_completo = DatasetDict(
    {
        "train": dataset_entrenamiento_validacion["train"],
        "validation": dataset_entrenamiento_validacion["test"],
        "test": dataset_test,
        "trainMasFrecuentes": dataset_entrenamiento_validacion_masFrecuentes["train"],
        "validationMasFrecuentes": dataset_entrenamiento_validacion_masFrecuentes["test"],
        "testMasFrecuentes": dataset_test_masFrecuentes
    })

dataset_completo

SyntaxError: ignored

In [ ]:
dataset_completo["train"][0]

In [ ]:
dataset_completo["train"][0]['text']

In [ ]:
dataset_completo["train"].features

In [ ]:
dataset_completo["test"].features

In [ ]:
dataset_completo.save_to_disk(ruta_dataset)